## This notebook is used for monitoring the model. It uses the latest images from the validated set of images and recomputes the Model KPIs.

### Getting images and their validated jsons from CosmosDB

#### Importing all libraries

In [ ]:
%load_ext dotenv
%dotenv ../services/environments/.env.secrets

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
%load_ext snakeviz
import holoviews as hv
hv.extension('bokeh')
# Import standard libs
import docx
import logging
import os
import pickle
import pycocotools
import random
import regex
import sys
import shutil
from collections import OrderedDict
from copy import deepcopy
from itertools import accumulate
from operator import add
from pathlib import Path
from pprint import pprint
from typing import *
from uuid import uuid4


# import first party libs
import cv2
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
print(torch.__version__, torch.cuda.is_available())
from PIL import Image
from skimage.measure import label, regionprops, find_contours
from tqdm.auto import tqdm


# import detectron2 utilities
import detectron2
from detectron2.engine import DefaultTrainer
from detectron2.utils.logger import setup_logger

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode, Boxes
from detectron2.data import transforms as T
from detectron2.data import build_detection_train_loader, build_detection_test_loader, DatasetMapper
from detectron2.data import detection_utils as utils

from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.config.config import CfgNode
from detectron2.engine import hooks

from detectron2.utils import comm
from detectron2.structures import Instances
from detectron2.evaluation import COCOEvaluator, DatasetEvaluator, DatasetEvaluators
from detectron2.modeling import GeneralizedRCNNWithTTA

from detectron2.checkpoint import DetectionCheckpointer

from detectron2.evaluation import verify_results

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", 100)
setup_logger()
cv2.setNumThreads(0)

# Append codebase path
sys.path.append("../src/")
from vxgioq_cfu_counting_ml.api.data_models import Predictions, Region, ShapeAttributes

from vxgioq_cfu_counting_ml.training.dataset_mapper import CFUDatasetMapper
from vxgioq_cfu_counting_ml.training.data_analysis import get_dataset_stats, check_retest_models, compute_kpis, get_data_compute_predictions_save_zip_upload_images_names_predictions_count, construct_model_info, predict_images
from vxgioq_cfu_counting_ml.training.matched_data import MATCHES
from vxgioq_cfu_counting_ml.training.organize_data import get_paths_test_images, get_paths_train_images, count_files, get_paths_positive_unlabelled_images, via_to_detectron2, custom_train_test_split, generate_train_test, gsk_original_to_detectron2, negative_to_detectron2
from vxgioq_cfu_counting_ml.training.trainer import CFUTrainer
from vxgioq_cfu_counting_ml.utils.loader import get_best_model_name,download_unzip_best_model


from vxgioq_cfu_counting_ml.utils.azure import (
download_image,
delete_object_from_blob,
download_load_models_kpis,
download_unzip_clean_zip,
get_cosmos_mongo_collection,
get_cosmos_mongo_collection_phase2,
get_file_from_blob,
list_data_on_blob,
list_models_on_blob,
upload_file_to_blob,
download_unzip_clean_data,
download_unzip_clean_model,
write_zip_upload_models_kpis,
zip_upload_data_to_blob,
zip_upload_file_folder_to_blob,
zip_upload_model_to_blob,
zip_upload_predictions_to_blob,
ACCOUNT_URL,
CONTAINER,
DATASET_DETECTRON2_NAME,
DATASETS_QUANTILES_NAME,
MODEL_CONFIG_NAME,
MODELS_KPIS_NAME,
MODELS_QUANTILES_NAME,
MODEL_WEIGHTS_NAME,
PATH_DATA_ON_DISK,
PATH_MODELS_ON_DISK,
PATH_PREDICTIONS_ON_DISK
)
from vxgioq_cfu_counting_ml.utils.conf import Backbone, Device, clean_output_dir_add_cfg, delete_files, delete_folders, prepare_cfu_detectron2_config_from_detectron2_model_zoo, prepeare_cfu_detectron_config_from_existing_model
from vxgioq_cfu_counting_ml.utils.prediction_intervals import pad_arrays, scores_to_distributions, generate_pi_dataset, DataType, train_quantile_regressor, HPSpace, verify_quantiles_predictions, BinningOption, DistributionFunctionType, scores_to_compiled_cdfs
from vxgioq_cfu_counting_ml.utils.convert import enhance_detectron2_dataset, extract_challenging_images, load_scan4000_excel,  extract_all_from_scan4000_report
from vxgioq_cfu_counting_ml.utils.custom_datasets import func_dataset, mask_2_polygon
from vxgioq_cfu_counting_ml.utils.display import compare_scan4000_and_dl_images, display_batch, display_pi, display_dataset, display_enhanced_dataset, display_features_importance, display_violin, NormalizeOption, save_examples
from vxgioq_cfu_counting_ml.utils.evaluator import CFUEvaluator
from vxgioq_cfu_counting_ml.utils.loader import load_model, get_best_model_name, get_best_model, load_images
from vxgioq_cfu_counting_ml.utils.metrics import mse, mae, mape, calibration
from vxgioq_cfu_counting_ml.utils.predictor import CFUPredictor
from vxgioq_cfu_counting_ml.utils.types import *

In [ ]:
import glob
from vxgioq_cfu_counting_ml.utils.azure import download_image
from PIL import Image

In [ ]:
import time
from datetime import datetime, timedelta, timezone
from typing import Any, Callable, Dict, List, Union
import sys
import arrow
from timeloop import Timeloop
import pandas as pd

In [ ]:
from vxgioq_cfu_counting_ml.api.data_models import (
    CFUInput,
    CFUOutput,
    ImageOutput,
    Predictions,
    Region,
    ShapeAttributes,
)
from vxgioq_cfu_counting_ml.utils import conf
from vxgioq_cfu_counting_ml.utils.azure import download_image, get_cosmos_mongo_collection,download_unzip_clean_model,get_blob_service_client,_zip_file
from vxgioq_cfu_counting_ml.utils.custom_datasets import mask_2_polygon
from vxgioq_cfu_counting_ml.utils.predictor import get_best_cfu_model, CFUPredictor

In [ ]:
import json

import tempfile
from pickle import HIGHEST_PROTOCOL  # noqa: S403
from typing import Any, List, Optional, Sequence, Tuple, Union

import joblib
from detectron2.structures import BoxMode
from pydantic import ValidationError
from skimage.measure import find_contours, label
from sklearn.utils import _safe_indexing
from sklearn.utils.validation import indexable

sys.path.append("VXGIOQ-CFU-Counting-ML/src/")

from vxgioq_cfu_counting_ml.utils.types import (
    DatasetVIAFormat,
    Detectron2Annotation,
    Detectron2DatasetElement,
    VIACircle,
    VIAEllipse,
    VIAPolygon,
)

In [ ]:
dishes_collection2 = get_cosmos_mongo_collection_phase2()

#### User needs to enter the criteria for getting images, if last_month is set to True, this will get all images from last one month. In other case, User can enter Start date and end date. There is a max limit set to restrict the max number of images downloaded.

In [ ]:
last_month=False  #if we need images from the past 30 days
max_no_of_images =400   #the number of images selected
days_range = 30  #number of days, if last_month set to True, the range is curr Date - days_range
no_of_days = timedelta(days_range)

path_jsons = '../data/new_jsons/'
path_data = '../data'

In [ ]:
if(last_month):
    #selects the start date 30 days before the current date
    ending_date = datetime.now()
    starting_date = ending_date - no_of_days

else:
    #for a particular timespan
    starting_date = datetime(2022,2,20)  # year,month,day format
    ending_date = datetime(2022,3,11)
    #ending_date = starting_date + no_of_days

In [ ]:
print(starting_date)
print(ending_date)

In [ ]:
total_jsons = list(dishes_collection2.find({'predictions.image.predictions_datetime':{"$gte":starting_date, "$lte":ending_date}}))
print(len(total_jsons))
processed_images=len(total_jsons)

In [ ]:
jsons = list(dishes_collection2.find({ "$and": [ { "is_validated":1 }, { 'predictions.image.predictions_datetime': {"$gte":starting_date, "$lte":ending_date} } ] }).limit(max_no_of_images))
print(len(jsons))

In [ ]:
print(f'Taking {len(jsons)} images and their jsons between {starting_date} and {ending_date}...')

### Creating the folders for the images and the jsons

In [ ]:
if(not os.path.exists(path_data)):
    os.makedirs(path_data)
    print(f'Creating folder for data in {path_data}')

In [ ]:
path_imgs = os.path.join(path_data,'new_images')

In [ ]:
if(not os.path.exists(path_jsons)):
    os.makedirs(path_jsons)
    print(f'Creating folder for jsons in {path_jsons}')
else:
    shutil.rmtree(path_jsons)
    os.makedirs(path_jsons)
    print(f'Deleting old folder and creating new folder for jsons in {path_jsons}')

if(not os.path.exists(path_imgs)):
    print(f'Creating folder for images in {path_imgs}')
    os.makedirs(path_imgs)
else:
    shutil.rmtree(path_imgs)
    os.makedirs(path_imgs)
    print(f'Deleting old folder and creating new folder for images in {path_imgs}')

In [ ]:
for i in range(len(jsons)):
    image_url = jsons[i]['image_original_url']
    x = download_image(image_url)
    img = Image.fromarray(x)
    img_name  = jsons[i]['sample_id']+'.png'
    img.save(os.path.join(path_imgs,img_name))
    
    json_name = jsons[i]['sample_id']+'.json'
    with open(path_jsons+json_name,'w') as dw:
        jsons[i].pop("_id")
        jsons[i]["predictions"]['image']['predictions_datetime'] = jsons[i]["predictions"]['image']['predictions_datetime'].strftime('%Y-%m-%dT%H:%M:%S.%sZ')
        jsons[i]["scanned_datetime"] = jsons[i]["scanned_datetime"].strftime('%Y-%m-%dT%H:%M:%S.%sZ')
        json.dump(jsons[i],dw)

## Getting total count, mold count , cfu count from jsons

In [ ]:
jsonNames = []
jsonData = []
for json_name in glob.glob(os.path.join(path_jsons,'*')):
    jsonNames.append(json_name)


for r,i in enumerate(jsonNames):
    with open(i) as x:
        f = json.load(x)
        jsonData.append(f)
len(jsonData)

In [ ]:
compData = []
n=0
for i in range(len(jsonData)):
    predMolds = 0
    predCfus = 0
    corrMolds = 0
    corrCfus = 0
    
    for j in jsonData[i]['predictions']['image']['regions']:
        if(j['shape_attributes']['category_id']==0):
            predCfus+=1
        elif(j['shape_attributes']['category_id']==1):
            predMolds+=1
        
    for j in jsonData[i]['corrections'][-1]['image']['regions']:
        if(j['shape_attributes']['category_id']==0):
            corrCfus+=1
        elif(j['shape_attributes']['category_id']==1):
            corrMolds+=1
    n+=1
    data = []
    data.append(n)
    data.append(jsonData[i]['sample_id'])
    data.append(jsonData[i]['predictions']['image']['predictions_cfu_count'])
    data.append(jsonData[i]['corrections'][-1]['image']['validated_cfu_count'])
    
    data.append(predCfus)
    data.append(corrCfus)

    data.append(predMolds)
    data.append(corrMolds)
    
    compData.append(data)

In [ ]:
results = pd.DataFrame(compData,columns=['Image No.','Sample ID','Predicted Total Count','Actual Total Count',
                                'Predicted CFU Count','Actual CFU Count',
                               'Predicted Mold Count','Actual Mold Count'])

In [ ]:
results

In [ ]:
cfu_evaluator = CFUEvaluator()
cfu_evaluator.gt_counts = results['Actual Total Count'].to_list()
cfu_evaluator.pred_counts = results['Predicted Total Count'].to_list()
kpis_dl = cfu_evaluator.evaluate()
print(cfu_evaluator.gt_counts,"- Actual counts")
print(cfu_evaluator.pred_counts,"- Predicted counts")

In [ ]:
print(kpis_dl)

In [ ]:
totaldev = 0
totaldev2 = 0
for i in range(len(results)):
    dev = abs(results['Predicted Total Count'][i]-results['Actual Total Count'][i])
    dev2 = abs(results['Predicted Total Count'][i]-results['Actual Total Count'][i])/results['Actual Total Count'][i]
    totaldev += dev
    totaldev2 += dev2
mae = totaldev/n
mape = (totaldev2/n)*100
print(mae)
print(mape)

In [ ]:
tableData = results

In [ ]:
results['Count match'] = np.where(results['Predicted Total Count']==results['Actual Total Count'],1,0)
results['CFU count match'] = np.where(results['Predicted CFU Count']==results['Actual CFU Count'],1,0)
results['Mold count match'] = np.where(results['Predicted Mold Count']==results['Actual Mold Count'],1,0)
results['False Negatives-Molds'] = np.where(results['Predicted Mold Count']<results['Actual Mold Count'],results['Actual Mold Count']-results['Predicted Mold Count'],0)
results['False Positives-Molds'] = np.where(results['Predicted Mold Count']>results['Actual Mold Count'],results['Predicted Mold Count']-results['Actual Mold Count'],0)

In [ ]:
print(f'Number of images : {n}')

In [ ]:
results

In [ ]:
countAcc =round(results['Count match'].sum()/n,2)
cfuAcc = round(results['CFU count match'].sum()/n,2)
moldAcc = round(results['Mold count match'].sum()/n,2)
countMis = n-results['Count match'].sum()
cfuMis = n-results['CFU count match'].sum()
moldMis = n-results['Mold count match'].sum()
fnMolds = results['False Negatives-Molds'].sum()
fpMolds = results['False Positives-Molds'].sum()

actualCfus = results['Actual CFU Count'].sum()
actualMolds = results['Actual Mold Count'].sum()

start = starting_date.strftime('%d-%B-%Y')
end = ending_date.strftime('%d-%B-%Y')
print(start,end)
timePeriod = ending_date.strftime('%d-%m-%Y')
print(timePeriod)

In [ ]:
resultsData =[[timePeriod,countAcc,cfuAcc,moldAcc,f'{countMis}/{n}',f'{cfuMis}/{n}',f'{moldMis}/{n}',
               f'{fnMolds} out of {actualMolds} actual molds',f'{fpMolds} out of {actualCfus} actual CFUs',
              round(mae,2),round(kpis_dl['main']['MAE (counts) 0'],2),round(kpis_dl['main']['MAE (counts) 1-5'],2),
               round(kpis_dl['main']['MAE (counts) 6-10'],2),round(kpis_dl['main']['MAPE (% counts) 11+'],2),round(mape,2)]]
resultsData

In [ ]:
df = pd.DataFrame(data=resultsData,columns=['Time period','Total Count Accuracy','Mold Count Accuracy','CFU Count Accuracy',
                                    'Total Count mismatches','Total CFU Count mismatches','Total Mold count mismatches','False Negatives-Molds','False Positives-Molds',
                                           'MAE (overall)','MAE (counts) 0', 'MAE (counts) 1-5', 'MAE (counts) 6-10', 'MAPE (% counts) 11+','MAPE (overall)'])

In [ ]:
df

In [ ]:
path_monitoring = '../monitoring_data'
if(not os.path.exists(path_monitoring)):
    os.makedirs(path_monitoring)
    print(f'Creating folder for monitoring excel file in {path_monitoring}')

In [ ]:
#if we have data for prev month as well and we want to append new data in csv file use this
download_unzip_clean_zip("model_monitoring_comp.csv.zip",path_monitoring,f'{ACCOUNT_URL}/machine-learning/monitoring',overwrite=True)
df.to_csv(os.path.join(path_monitoring,'model_monitoring_comp.csv'),mode='a',index=False,header=False)

In [ ]:
#zip_upload_file_folder_to_blob(os.path.join(path_monitoring,'model_monitoring_comp.csv'),url=f'{ACCOUNT_URL}/machine-learning/monitoring/model_monitoring_comp.csv.zip',overwrite=True)

In [ ]:
download_unzip_clean_zip("model_monitoring_comp.csv.zip",path_monitoring,f'{ACCOUNT_URL}/machine-learning/monitoring',overwrite=True)

In [ ]:
df = pd.read_csv(os.path.join(path_monitoring,'model_monitoring_comp.csv'))

In [ ]:
df = df.fillna('NA')

In [ ]:
df

In [ ]:
df2 = df[['MAE (overall)','MAE (counts) 0', 'MAE (counts) 1-5', 'MAE (counts) 6-10', 'MAPE (% counts) 11+','MAPE (overall)']]
df = df[['Time period','Total Count Accuracy','Mold Count Accuracy','CFU Count Accuracy',
                                    'Total Count mismatches','Total CFU Count mismatches','Total Mold count mismatches','False Negatives-Molds','False Positives-Molds']]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime,timedelta
import numpy as np
%matplotlib inline
fig,ax = plt.subplots(figsize=(7,5))

ax.set_title('Accuracy Trend',fontsize=14)
ax.set(xlabel='Time Period',ylabel='Accuracy')

xi = [datetime.strptime(date,'%d-%m-%Y') for date in df['Time period']]
x = [datetime.strftime(date,'%m-%Y') for date in xi]


ax.set_ylim(0,1)
ax.set_yticks(np.linspace(0,1,11))

ax.plot(x,df['Total Count Accuracy'],label='Total Count Accuracy' ,color='green', marker='o',linestyle='dashed',alpha=1)
ax.plot(x,df['CFU Count Accuracy'],label='CFU Count Accuracy' , color='blue', marker='o',linewidth=2,alpha=0.5)
ax.plot(x,df['Mold Count Accuracy'], label='Mold Count Accuracy' ,color='red', marker='o',linewidth=2,alpha=0.7)
plt.xticks(rotation=45)
plt.grid(color='gray',linestyle='dashed',linewidth=0.5)
plt.legend(loc='best')
plt.savefig(os.path.join(path_monitoring,'accuracyPlot.png'))
plt.show()

### Download Models KPIs

In [ ]:
# download and load the models kpis
models_kpis = download_load_models_kpis(overwrite=False)

### Download the best available model

In [ ]:
best_model_name = download_unzip_best_model(task="main", metric="MAE (counts) 6-10")

In [ ]:
print(best_model_name)

### Select the KPIs for the best model

In [ ]:
for i in range(len(models_kpis.models)):
    if best_model_name != models_kpis.models[i].name:
        models_kpis.models.pop(i)

In [ ]:
best_model_kpis = models_kpis.models[0]

In [ ]:
models_kpis

## Saving the metrics and plot in a word document

In [ ]:
doc = docx.Document()
title = doc.add_heading(f"Model Monitoring Metrics",2)
title.alignment = 1
doc.add_paragraph('-'*118)

doc.add_paragraph(f"Time Period between {start} and {end}")
doc.add_heading(f"Number of images processed: {processed_images}",3)
doc.add_heading(f"Number of validated images: {len(jsonData)}",3)

doc.add_heading(f"Number of images used to check the metrics: {len(jsonData)}",3)
doc.add_heading(f"Model Name: {best_model_name}",3)
doc.add_paragraph()

table1 = doc.add_table(rows=df.shape[0]+1,cols=df.shape[1],style='Light Grid Accent 2')

for j in range(df.shape[-1]):
    table1.cell(0,j).text = df.columns[j]

for i in range(df.shape[0]):
    for j in range(df.shape[-1]):
        table1.cell(i+1,j).text = str(df.values[i,j])
doc.add_page_break()
doc.add_picture(os.path.join(path_monitoring,'accuracyPlot.png'))

### Saving existing model KPIs in a document

In [ ]:
doc.add_page_break()
if(len(df2)==1):
    num_rows = 7
    num_columns = 3
else:
    num_rows = 7
    num_columns = 4
#doc.add_heading(f"Original Model KPIs for Model Name: {best_model_name}",3)
table = doc.add_table(rows=num_rows,cols=num_columns,style='Light Grid Accent 2')
table.cell(0,0).text = 'MODEL METRICS'
table.cell(1,0).text = 'MAE (counts) 0'
table.cell(2,0).text = 'MAE (counts) 1-5'
table.cell(3,0).text = 'MAE (counts) 6-10'
table.cell(4,0).text = 'MAPE (% counts) 11+'
table.cell(5,0).text = 'MAE (overall)'
table.cell(6,0).text = 'MAPE (% overall)'
  
table.cell(0,1).text = f'Original Model KPIs(on initial data)'
table.cell(1,1).text = str(round(best_model_kpis.kpis.main['MAE (counts) 0'],2))
table.cell(2,1).text = str(round(best_model_kpis.kpis.main['MAE (counts) 1-5'],2))
table.cell(3,1).text = str(round(best_model_kpis.kpis.main['MAE (counts) 6-10'],2))
table.cell(4,1).text = str(round(best_model_kpis.kpis.main['MAPE (% counts) 11+'],2))
table.cell(5,1).text = '-'
table.cell(6,1).text = '-'

### Saving recomputed model KPIs in a document

In [ ]:
x = len(df2)

In [ ]:
if(x==1):
    table.cell(0,2).text = f'Model KPIs(current month)'
    table.cell(1,2).text = str(df2['MAE (counts) 0'][0])
    table.cell(2,2).text = str(df2['MAE (counts) 1-5'][0])
    table.cell(3,2).text = str(df2['MAE (counts) 6-10'][0])
    table.cell(4,2).text = str(df2['MAPE (% counts) 11+'][0])
    table.cell(5,2).text = str(df2['MAE (overall)'][0])
    table.cell(6,2).text = str(df2['MAPE (overall)'][0])

else:
    table.cell(0,2).text = f'Model KPIs(previous month)'
    table.cell(1,2).text = str(df2['MAE (counts) 0'][x-2])
    table.cell(2,2).text = str(df2['MAE (counts) 1-5'][x-2])
    table.cell(3,2).text = str(df2['MAE (counts) 6-10'][x-2])
    table.cell(4,2).text = str(df2['MAPE (% counts) 11+'][x-2])
    table.cell(5,2).text = str(df2['MAE (overall)'][x-2])
    table.cell(6,2).text = str(df2['MAPE (overall)'][x-2])
    
    table.cell(0,3).text = f'Model KPIs(current month)'
    table.cell(1,3).text = str(df2['MAE (counts) 0'][x-1])
    table.cell(2,3).text = str(df2['MAE (counts) 1-5'][x-1])
    table.cell(3,3).text = str(df2['MAE (counts) 6-10'][x-1])
    table.cell(4,3).text = str(df2['MAPE (% counts) 11+'][x-1])
    table.cell(5,3).text = str(df2['MAE (overall)'][x-1])
    table.cell(6,3).text = str(df2['MAPE (overall)'][x-1])

doc.add_paragraph()
doc.add_paragraph("MAE (counts) 0 - Mean Absolute Error for dishes with 0 CFU")
doc.add_paragraph("MAE (counts) 1-5 - Mean Absolute Error for dishes with 1 to 5 CFUs")
doc.add_paragraph("MAE (counts) 6-10 - Mean Absolute Error for dishes with 6 to 10 CFUs")
doc.add_paragraph("MAPE (% counts) 11+ - Mean Absolute Percentage Error for dishes with 11+ CFUs")

## Saving detailed metrics for current set of data as an appendix in document

In [ ]:
from docx.shared import Pt,Inches
doc.add_page_break()
doc.add_section()
sections = doc.sections
sections[1].left_margin = Inches(0.05)
sections[1].right_margin = Inches(0.01)

appendixHeading = doc.add_heading('Detailed Metrics',3)
appendixHeading.alignment = 1
doc.add_paragraph('-'*165)
doc.add_paragraph()
doc.add_paragraph("In fields, 'Count match','CFU count match' and 'Mold count match'")
doc.add_paragraph("    1 = there is a match in counts")
doc.add_paragraph("    0 = there is mismatch in counts")

table2 = doc.add_table(rows=tableData.shape[0]+1,cols=tableData.shape[1],style='Light Grid Accent 2')
for j in range(tableData.shape[-1]):
    table2.cell(0,j).text = tableData.columns[j]

for i in range(tableData.shape[0]):
    for j in range(tableData.shape[-1]):
        table2.cell(i+1,j).text = str(tableData.values[i,j])
        
for cell in table2.columns[0].cells:
    cell.width = Inches(0.5)
    
for cell in table2.columns[1].cells:
    cell.width = Inches(1.0)
    
for row in table2.rows:
    for cell in row.cells:
        paragraphs = cell.paragraphs
        paragraph = paragraphs[0]
        run_obj = paragraph.runs
        run = run_obj[0]
        font = run.font
        font.size = Pt(9)
        
doc_name='MONITORING-'+end+'.docx'
doc.save(os.path.join(path_monitoring,doc_name))

## Uploading monitoring report to blob

In [ ]:
#upload_file_to_blob(os.path.join(path_monitoring,doc_name),url=f'{ACCOUNT_URL}/machine-learning/monitoring/'+doc_name,overwrite=True)

## Sending the Report as Word doc attachment through email


In [ ]:
import smtplib
from email.message import EmailMessage
from email.mime.base import MIMEBase 

In mailto we keep the mail id we want to send the email to and in mailfrom we keep the email id through which mail will be sent.

In [ ]:
mailto="ritika.x.koul@gsk.com"
#mailto=["ritika.x.koul@gsk.com","anirban.x.konar@gsk.com"]
mailfrom="ritika.x.koul@gsk.com"

In [ ]:
message="CFU Model monitoring job has completed."
message+="You can find the results attached here."

mailsubject="CFU Monitoring job completed."
msg=EmailMessage()
msg.set_content(message)
msg['Subject'] = mailsubject
msg['From'] = mailfrom
msg['To'] = mailto

with open(os.path.join(path_monitoring,doc_name), 'rb') as fp:
    data = fp.read()
    ctype = 'application/octet-stream'
    maintype, subtype = ctype.split('/', 1)
    msg.add_attachment(data, maintype=maintype, subtype=subtype, filename=doc_name)


server=smtplib.SMTP('localhost')
server.send_message( msg )
server.quit()